In [23]:
%pip install google-genai


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [24]:
%pip install dotenv


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [25]:
import json 
from google import genai
import os
import requests 
from dotenv import load_dotenv
import threading 
load_dotenv()

True

In [2]:
for key in os.environ:
        if key.startswith("GOOGLE_"):
                print(key, os.environ[key])

In [26]:

google_key = os.getenv("GOOGLE_API_KEY")
client = genai.Client(api_key=google_key)


In [ ]:
from pydantic import BaseModel
from typing import List, Optional


class Lesson(BaseModel):
    lesson: str
    duration_minutes: int
    topics: List[str]


class Unit(BaseModel):
    Unit_number: int
    title: str
    unit_description: str
    learning_objectives: List[str]
    lesson_outline: List[Lesson]
    assessment: str


class FinalAssessment(BaseModel):
    final_exam_description: str


class Course(BaseModel):
    title: str
    description: str
    estimated_duration_hours: int
    prerequisites: List[str]
    units: List[Unit]
    final_assessment: FinalAssessment


class Syllabus(BaseModel):
    course: Course

class LessonTopic(BaseModel):
    title: str

In [ ]:
# def get_syllabus(topic: str, difficulty: str):
#     response = client.models.generate_content(
#         model="gemini-2.0-flash",
#         contents=[
#             f"Generate a complete syllabus for a {difficulty} {topic} class.",
#             "Include lesson titles, and for each, provide Lesson Description,Learning Objectives, Lesson Outline, and Assessment.",
#             "End with a Final Exam section detailing topics that will be tested.",
#             "Return the output as structured JSON only.",
#         ],
#         config={
#             'response_mime_type': 'application/json',
#             'response_schema': Syllabus,
#         },
#     )
#     return response.text

'''uses the fields on the website'''

def get_syllabus(topic: str, difficulty: str, depth: str) -> str:
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=[
            f"Generate a complete syllabus for a {difficulty} class of depth {depth}.",
            f"The subject is: {topic}",
            "Include lesson titles, and for each, provide Lesson Description,Learning Objectives, Lesson Outline, and Assessment.",
            "End with a Final Exam section detailing topics that will be tested.",
            "Return the output as structured JSON only.",
        ],
        config={
            'response_mime_type': 'application/json',
            'response_schema': Syllabus,
        },
    )
    return response.text

In [29]:
syllabus = get_syllabus('machine learning','easy')

In [30]:
print(syllabus)

{
  "course": {
    "title": "Introduction to Machine Learning",
    "description": "This course provides a gentle introduction to the fundamental concepts and techniques of machine learning. Students will learn about supervised and unsupervised learning algorithms, model evaluation, and practical considerations for applying machine learning in real-world scenarios. No prior machine learning experience is required.",
    "estimated_duration_hours": 45,
    "prerequisites": [
      "Basic programming skills (e.g., Python)",
      "Basic understanding of linear algebra and statistics"
    ],
    "units": [
      {
        "Unit_number": 1,
        "title": "Introduction to Machine Learning",
        "unit_description": "This unit introduces the core concepts of machine learning, its different types, and the machine learning workflow.",
        "learning_objectives": [
          "Define machine learning and its applications.",
          "Distinguish between supervised, unsupervised, and r

In [ ]:
from pydantic import BaseModel
from typing import List, Optional


class Lesson(BaseModel):
    lesson: str
    duration_minutes: int
    topics: List[str]


class LessonContent(BaseModel):
    readings: List[str]
    examples: List[str]
    exercises: List[str]
    assessments: List[str]
    additional_resources: List[str]  # NEW FIELD


class Unit(BaseModel):
    unit_number: int
    title: str
    unit_description: str
    learning_objectives: List[str]
    lesson_outline: List[Lesson]
    assessment: str
    content: Optional[LessonContent] = None


class FinalAssessment(BaseModel):
    final_exam_description: str
    final_project_description: str


class Course(BaseModel):
    title: str
    description: str
    estimated_duration_hours: int
    prerequisites: List[str]
    lesson: List[Unit]
    final_assessment: FinalAssessment


class Syllabus(BaseModel):
    course: Course


class LessonTopic(BaseModel):
    title: str


In [201]:
import re
import requests

# Function to validate a URL using requests
def validate_url(url: str) -> bool:
    try:
        response = requests.head(url, allow_redirects=True, timeout=5)
        if response.status_code >= 400 or response.status_code == 405:
            response = requests.get(url, allow_redirects=True, timeout=8)
        return response.status_code < 400
    except requests.RequestException:
        return False

# Function to extract and validate URLs from the 'Additional Resources' section of lesson content
def extract_and_validate_additional_resources(text: str, unit_title: str) -> str:
    updated_text = ""
    inside_resources = False

    for line in text.splitlines():
        if line.strip().lower().startswith("additional resources"):
            inside_resources = True
            updated_text += f"Unit: {unit_title}\n"  # Add the unit title before the resources section
            updated_text += line + "\n"  # Keep the "Additional Resources" header
            continue

        if inside_resources:
            if not line.strip():  # End of the section
                updated_text += line + "\n"
                break

            match = re.search(r'https?://\S+', line)
            if match:
                url = match.group(0)
                if validate_url(url):
                    updated_text += line + "\n"  # Keep valid URLs
            # If the URL is invalid, we don't add it to the `updated_text`
        else:
            updated_text += line + "\n"  # Keep everything outside of "Additional Resources"

    return updated_text

In [211]:
def generate_lesson_content(lesson_title: str, learning_objectives: List[str], lesson_outline: List[Lesson],unit_title: str):
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=[
            f"Generate detailed content for the lesson titled '{lesson_title}' with the following learning objectives: {', '.join(learning_objectives)}.",
            "Provide readings to help learn the material, practical examples with explanations, exercises for practice, and assessments for evaluation.",
            "The readings should be clear and comprehensive.",
            "The examples should demonstrate the concepts effectively.",
            "The exercises should be practice-oriented and the assessments should be auto-gradable.",
            "Include a section at the end titled 'Additional Resources:' with 6 helpful and relevant resource links (one per line, starting with a dash).",
            "Return the content as plain text, separated into sections for 'Readings', 'Examples', 'Exercises', and 'Assessments'."
        ],
        config={
            'response_mime_type': 'text/plain', 
        }
    )
    updated_text = extract_and_validate_additional_resources(response.text,unit_title)
    return updated_text

In [212]:
import json
import threading
import time

# Threaded lesson content generation
def generate_lesson_threaded(lesson, unit_learning_objectives, index, all_content, unit_title):
    try:
        time.sleep(0.5)
        lesson_content_text = generate_lesson_content(
            lesson_title=lesson["lesson"],
            learning_objectives=unit_learning_objectives,
            lesson_outline=[lesson],
            unit_title=unit_title  # 👈 Pass unit title here
        )
        if lesson_content_text:
            all_content[index] = f"\n\n=== {unit_title} ===\n\n{lesson_content_text}"
    except Exception as e:
        all_content[index] = f"Error generating content for lesson {lesson['lesson']}: {e}"

# Main course generator
def generate_full_course(syllabus_json_str: str) -> str:
    try:
        syllabus_data = json.loads(syllabus_json_str)
    except Exception as e:
        raise ValueError(f"Invalid JSON input. Error: {e}")

    # Adjusted for your JSON schema: "lessons" holds units
    total_lessons = sum(len(unit["lesson_outline"]) for unit in syllabus_data["course"]["lessons"])
    has_final_exam = "final_assessment" in syllabus_data["course"] and \
                     "final_exam_description" in syllabus_data["course"]["final_assessment"]

    all_content = [None] * (total_lessons + int(has_final_exam))
    threads = []
    index = 0

    for unit in syllabus_data["course"]["lessons"]:
        unit_title = f"UNIT {unit['unit_number']}: {unit['title']}"
        for lesson in unit["lesson_outline"]:
            thread = threading.Thread(
                target=generate_lesson_threaded,
                args=(lesson, unit["learning_objectives"], index, all_content, unit_title)
            )
            threads.append(thread)
            thread.start()
            index += 1

    # Final Exam
    if has_final_exam:
        final_exam_description = syllabus_data["course"]["final_assessment"]["final_exam_description"]

        def generate_final_exam(index, all_content):
            try:
                time.sleep(0.5)
                response = client.models.generate_content(
                    model="gemini-2.0-flash",
                    contents=[
                        "Generate a final exam for the course based on the following description.",
                        final_exam_description,
                        "Include a variety of question types such as multiple choice, short answer, and coding tasks.",
                        "Make the exam comprehensive and clearly structured.",
                        "Return this as plain text."
                    ],
                    config={"response_mime_type": "text/plain"}
                )
                all_content[index] = "\n\n=== FINAL EXAM ===\n\n" + response.text
            except Exception as e:
                all_content[index] = f"Error generating final exam: {e}"

        thread = threading.Thread(
            target=generate_final_exam,
            args=(index, all_content)
        )
        threads.append(thread)
        thread.start()

    # Wait for all threads to complete
    for thread in threads:
        thread.join()

    return "\n".join(content for content in all_content if content)

In [213]:
c = generate_full_course(syllabus)

In [214]:
print(c)



=== UNIT 1: Introduction to Machine Learning ===

Here's the lesson content, separated into the requested sections:

**Readings**

**What is Machine Learning?**

Machine learning (ML) is a field of computer science that gives computer systems the ability to learn from data without being explicitly programmed. Instead of writing specific instructions for every possible situation, ML algorithms learn patterns and relationships in data to make predictions or decisions. Think of it like teaching a dog: you don't tell it *exactly* how to sit using complex rules, you reward it when it sits correctly and discourage it when it doesn't. ML algorithms work similarly, adjusting their internal parameters based on the data they're exposed to.

**Scope of Machine Learning:**

Machine learning has a broad scope and is applicable to numerous domains, including:

*   **Image Recognition:** Identifying objects in images (e.g., faces, cars, animals).
*   **Natural Language Processing (NLP):** Understan

In [198]:
import json

def test_lesson_content_json(syllabus_json_str: str) -> str:
    # Parse input
    all_content = ""
    try:
        syllabus_data = json.loads(syllabus_json_str)
    except Exception as e:
        raise ValueError(f"Invalid JSON input. Error: {e}")

    # First unit and first lesson
    first_unit = syllabus_data["course"]["units"][0]
    first_lesson = first_unit["lesson_outline"][0]

    # Generate lesson content (raw text)
    lesson_content_text = generate_lesson_content(
        lesson_title=first_lesson["lesson"],
        learning_objectives=first_unit["learning_objectives"],
        lesson_outline=[first_lesson]
    )
    if lesson_content_text:
        all_content += lesson_content_text.strip() + "\n"
    # Print the content for debugging
    # Return just the raw content string
    return all_content
